In [6]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

# os.environ["OPENAI_API_KEY"] = #
client = OpenAI()


## Test

In [16]:
response = client.chat.completions.create(
model="gpt-4o-mini",
messages=[
    {"role": "system", "content": "You are a psychologist and an art critic. Be concise and creative."}, {"role": "user", "content": [{"type": "text", "text": "Here is a list of artworks that a users liked.\nComplete the user profile based on these images. \nAnswer with age (exact number), sex (F or M) and a creative sentence describing the user in general (culture, location, work, aestehtics, emotions...).\nFormat the response as a JSON object with the keys age, gender, and description."}, {"type": "image_url", "image_url": {"url": "https://d32dm0rphc51dk.cloudfront.net/rm-3xIq2CMHaslmMcDQyYg/medium.jpg"}}, {"type": "image_url", "image_url": {"url": "https://d32dm0rphc51dk.cloudfront.net/BeHlOZJPnme2pTs1T-CL2w/medium.jpg"}}, {"type": "image_url", "image_url": {"url": "https://d32dm0rphc51dk.cloudfront.net/2VgaThXZd_AK3IH2-dwe2w/medium.jpg"}}]}
],
max_tokens=300,
)

In [17]:
print(response)

ChatCompletion(id='chatcmpl-Afo5buqQJxYXARHtCvypyzgMHAgMe', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n{\n  "age": 45,\n  "gender": "F",\n  "description": "A culturally rich individual who thrives in historical narratives, blending an appreciation for art and geography with a penchant for the stories of the past, likely residing in a vibrant European city."\n}\n```', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1734527355, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_39a40c96a0', usage=CompletionUsage(completion_tokens=60, prompt_tokens=25600, total_tokens=25660, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


# Run the batch

In [18]:
batch_input_file = client.files.create(
  file=open("../03-generate-user-descriptions/user_batch_0-15_000.jsonl", "rb"),
  purpose="batch"
)
batch_input_file_id = batch_input_file.id

print(batch_input_file_id)

file-9n6nB4ugu3v1oHp6xah4zB


In [19]:
# create the  batch
batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "User descriptions based on artworks. Part 0-15000"
    }
)

Batch(id='batch_6762c9c4f80c8190b53b5c9ca1709676', completion_window='24h', created_at=1734527429, endpoint='/v1/chat/completions', input_file_id='file-9n6nB4ugu3v1oHp6xah4zB', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1734613829, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'User descriptions based on artworks. Part 0-15000'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [20]:
# check status of batch
from openai import OpenAI
client = OpenAI()

client.batches.retrieve('batch_6762c9c4f80c8190b53b5c9ca1709676')

Batch(id='batch_6762c9c4f80c8190b53b5c9ca1709676', completion_window='24h', created_at=1734527429, endpoint='/v1/chat/completions', input_file_id='file-9n6nB4ugu3v1oHp6xah4zB', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='token_limit_exceeded', line=None, message='Enqueued token limit reached for gpt-4o-mini in organization org-1u7PipOgiHmvwfH7N16VkhuJ. Limit: 20,000,000 enqueued tokens. Please try again once some in_progress batches have been completed.', param=None)], object='list'), expired_at=None, expires_at=1734613829, failed_at=1734527433, finalizing_at=None, in_progress_at=None, metadata={'description': 'User descriptions based on artworks. Part 0-15000'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

# list all batches

In [21]:
client.batches.list(limit=10)

SyncCursorPage[Batch](data=[Batch(id='batch_6762c9c4f80c8190b53b5c9ca1709676', completion_window='24h', created_at=1734527429, endpoint='/v1/chat/completions', input_file_id='file-9n6nB4ugu3v1oHp6xah4zB', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='token_limit_exceeded', line=None, message='Enqueued token limit reached for gpt-4o-mini in organization org-1u7PipOgiHmvwfH7N16VkhuJ. Limit: 20,000,000 enqueued tokens. Please try again once some in_progress batches have been completed.', param=None)], object='list'), expired_at=None, expires_at=1734613829, failed_at=1734527433, finalizing_at=None, in_progress_at=None, metadata={'description': 'User descriptions based on artworks. Part 0-15000'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0)), Batch(id='batch_6762c3db25fc8190aed0554fea2950d2', completion_window='24h', created_at=1734525915, endpoint

# Cancel batch

In [15]:
client.batches.cancel("batch_6762c3db25fc8190aed0554fea2950d2")

ConflictError: Error code: 409 - {'error': {'message': "Cannot cancel a batch with status 'failed'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

## Retrieve the results

In [ ]:
from openai import OpenAI
import json
client = OpenAI()

file_response = client.files.content('file-HQ3wv1ACMkxJXLkKvGnAST')
reponse = json.dumps(file_response.text)
output_name='artworks_results.jsonl'
try:
    with open(output_name,"w", encoding="utf-8") as outfile:
        outfile.write(json.loads(reponse))
except Exception as e:
        print(f"An error occurred: {e}")


In [20]:
import json
import pandas as pd

csv_file = '../data/artwork_results.csv'
data = []
try:
    # Load the JSON file
    with open("../data/artworks_results.jsonl", "r") as file:
        for line in file:
            data.append(json.loads(line))
    print(data[0].get("response",{}).get("body", {})
                .get("choices", [{}])[0]
                .get("message", {})
                .get("content",""))
    csv_data = pd.DataFrame([
        {
            "id": item.get("custom_id"),
            "description": (
                item.get("response",{}).get("body", {})
                .get("choices", [{}])[0]
                .get("message", {})
                .get("content","")
            )
        }
        for item in data
    ])

    # Save to CSV
    csv_data.to_csv(csv_file, index=False)
    print(f"CSV file saved at: {csv_file}")
    
   

except Exception as e:
    print(f"An error occurred: {e}")

The image is rich in golds and soft pastels, creating a warm, harmonious atmosphere. The mood is intimate and tender, evoking feelings of love and closeness. The aesthetic features intricate patterns and floral motifs, reminiscent of the Art Nouveau style. Overall, it radiates a sense of tranquility and beauty.
CSV file saved at: ../data/artwork_results.csv
